In [5]:
!pip install pandas requests

In [6]:
import pandas as pd

data = [
    {
        "id": 1,
        "prompt": "Summarize the clinical presentation and plan for a 65-year-old with type 2 diabetes and hypertension.",
        "key_facts": "type 2 diabetes|hypertension|management|follow-up"
    },
    {
        "id": 2,
        "prompt": "Write a brief medical summary for a patient presenting with chest pain and shortness of breath.",
        "key_facts": "chest pain|shortness of breath|differential|evaluation"
    }
]

df = pd.DataFrame(data)
df.to_csv("prompts.csv", index=False)
df


,id,prompt,key_facts
0,1,Summarize the clinical presentation and plan f...,type 2 diabetes|hypertension|management|follow-up
1,2,Write a brief medical summary for a patient pr...,chest pain|shortness of breath|differential|ev...


In [8]:
mock_responses = {
    1: "The patient has type 2 diabetes and hypertension. Management includes medication optimization, lifestyle changes, and routine follow-up.",
    2: "The patient presents with chest pain and shortness of breath. Evaluation should include ECG, labs, and imaging to assess cardiac and pulmonary causes."
}


In [9]:
import re

def normalize(text):
    return re.sub(r"\s+", " ", text.lower().strip())

def fact_coverage(response, facts):
    r = normalize(response)
    hits = sum(1 for f in facts if normalize(f) in r)
    return hits / max(len(facts), 1)

def coherence_score(response):
    words = response.split()
    score = 1.0
    if len(words) < 20:
        score -= 0.3
    if response.count(".") < 2:
        score -= 0.2
    return max(0.0, score)

results = []

for _, row in df.iterrows():
    rid = row["id"]
    response = mock_responses[rid]
    facts = row["key_facts"].split("|")

    results.append({
        "id": rid,
        "fact_coverage": fact_coverage(response, facts),
        "coherence_score": coherence_score(response)
    })

results_df = pd.DataFrame(results)
results_df


,id,fact_coverage,coherence_score
0,1,1.00,0.7
1,2,0.75,1.0


In [10]:
results_df.to_csv("metrics.csv", index=False)

!ls


metrics.csv  prompts.csv  sample_data
